The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import PyPep as pep
cea = pep.PyPep()

cea.fuel = "CH4:1"
cea.ox = "N2O:1"
cea.OF = 6
cea.fuel_TP = None, 3e5
cea.ox_TP = 280, 3e5

cea.solve()

cea()

cea.T


  gri30:

       temperature   2770.2 K
          pressure   2.9962e+05 Pa
           density   0.27811 kg/m^3
  mean mol. weight   21.379 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        9.1185e+05        1.9495e+07  J
   internal energy        -1.655e+05       -3.5382e+06  J
           entropy             12185        2.6051e+05  J/K
    Gibbs function       -3.2844e+07       -7.0218e+08  J
 heat capacity c_p            1912.7             40891  J/K
 heat capacity c_v            1523.7             32577  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2          0.016594           0.17597           -21.219
                 H         0.0008472          0.017969           -10.609
                 O        0.00025946        0.00034671           -18.

2770.2012385600538

In [2]:
import cantera as ct

gas = ct.Solution("gri30.yaml")
gas()


  gri30:

       temperature   300 K
          pressure   1.0133e+05 Pa
           density   0.081894 kg/m^3
  mean mol. weight   2.016 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy             26469             53361  J
   internal energy       -1.2108e+06        -2.441e+06  J
           entropy             64910        1.3086e+05  J/K
    Gibbs function       -1.9447e+07       -3.9204e+07  J
 heat capacity c_p             14311             28851  J/K
 heat capacity c_v             10187             20536  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2                 1                 1           -15.717
     [  +52 minor]                 0                 0  



In [30]:
class A:
    def __init__(self):
        self._x = 0
    @property
    def X(self): return self._x
    @X.setter
    def X(self, value):
        self._x = value

class B(A):
    def __init__(self):
        self._x = 5


i = A()

i.X = 10
i.X

i2 = B()

i2.X = 10
i2.X

10

In [4]:
# Reload modules automatically on change
%load_ext autoreload
%autoreload 2
import PyPep as pep
import cantera as ct

gas = ct.Solution("gri30.yaml")
solution = pep.Solution()
fuel = pep.Phase(solution())
ox = pep.Phase(solution())

fuel.comp = "CH4:1"
ox.comp = "O2:1"

fuel.TP = 300, 3e5
ox.TP = 250, 4e5

solution.solve(fuel, ox)

solution()




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

  gri30:

       temperature   250 K
          pressure   4e+05 Pa
           density   6.1576 kg/m^3
  mean mol. weight   31.998 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy            -44046       -1.4094e+06  J
   internal energy       -1.0901e+05        -3.488e+06  J
           entropy            5893.4        1.8858e+05  J/K
    Gibbs function       -1.5174e+06       -4.8553e+07  J
 heat capacity c_p            912.08             29185  J/K
 heat capacity c_v            652.24             20870  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2                 1                 1           -23.359
     [  +52 minor]        2.5181e-50        5.0362e-50  

